In [21]:
%store -r city_coord
%store -r air_coord
%store -r office_info
%store -r office_coords
%store -r of_size
%store -r of_ref

import mapping_functions as M
import ranking_functions as R
import location_functions as L
import mongo

import pandas as pd
from IPython.display import HTML


dbName = 'companies'

db,start = mongo.connectCollection(dbName,'startups')
db,old_comp = mongo.connectCollection(dbName,'old_comp')
db,starbucks = mongo.connectCollection(dbName,'starbucks')
db,schools = mongo.connectCollection(dbName,'schools')
db,night = mongo.connectCollection(dbName,'night')
db,vegan = mongo.connectCollection(dbName,'vegan')

In [22]:
names = ["Old companies","Starbucks","Night bars","Schools","Startups","Vegan restaurant","Airport","Offices"]  
collections = [old_comp,starbucks,night,schools,start,vegan]
colors = ["red","darkgreen","purple","lightblue","green","orange"]

In [23]:
M.mapResults(city_coord,13,names,colors,collections,air_coord,office_coords,"initial_map")

In [24]:
weight = []
for n in names[:6]:
    weight.append(int(input(f"Introduce the weight of {n} (1-5): ")))

Introduce the weight of Old companies (1-5): 5
Introduce the weight of Starbucks (1-5): 4
Introduce the weight of Night bars (1-5): 3
Introduce the weight of Schools (1-5): 2
Introduce the weight of Startups (1-5): 4
Introduce the weight of Vegan restaurant (1-5): 1


In [25]:
points,adresses,air_dist = R.pointsFunction(collections,office_coords,office_info,air_coord,weight)

Creamos un nuevo Dataframe con la informacion a cerca de cada una de las oficinas donde podriamos potencialmente colocar la empresa:
    - Direccion
    - Tamaño de la oficina
    - Puntuacion en el ranking total
    - Distancia a startups, starbucks, compañias antiguas, aeropuerto, colegios y bares mas cercanos
    - Coordenadas
    - Referencia

In [26]:
url_1 = "https://42floors.com/"

df_offices = pd.DataFrame(columns=['Adress','Square Feet','Punctuation','Startups','Starbucks',
                                   'Old companies','Schools','Night bars','Vegan Rest','Airport','Location','Reference'])
 
for index in range(len(points)):
    total = sum(points[index][1:])-points[index][0]
    df_offices = df_offices.append({'Adress':adresses[index],'Punctuation':total,
                                    'Starbucks':points[index][1],'Schools':points[index][3],'Startups':points[index][4],
                                    'Old companies':-points[index][0],'Night bars':points[index][2],
                                    'Square Feet':of_size[index],'Location':office_coords[index],
                                   'Reference':url_1+of_ref[index],'Airport':air_dist[index],'Vegan Rest':points[index][5]},ignore_index=True)

In [27]:
df = df_offices.sort_values(by=['Punctuation'],ascending=False)
df.to_csv('../output/office-location.csv')
df = df.reset_index(drop=True)
df

,Adress,Square Feet,Punctuation,Startups,Starbucks,Old companies,Schools,Night bars,Vegan Rest,Airport,Location,Reference
0,"255 State St, Boston, MA 02109, USA","19,940",18.75,9.0,10.0,-8.75,1.5,6.00,1.00,3.511837054493131 km,"[-71.0513553, 42.3593857]",https://42floors.com//us/ma/boston/255-state-s...
1,"40 Court St, Boston, MA 02108, USA","4,000",18.50,10.0,10.0,-12.50,1.0,7.50,2.50,4.16538866183103 km,"[-71.0592945, 42.3588496]",https://42floors.com//us/ma/boston/40-court-st...
2,"5 Broad St, Boston, MA 02110, USA","11,050",17.75,9.0,10.0,-11.25,1.5,6.75,1.75,3.8058028687439767 km,"[-71.0548501, 42.3588513]",https://42floors.com//us/ma/boston/75-state-st...
3,"38 Chauncy St, Boston, MA 02111, USA","6,560",17.25,10.0,9.0,-12.50,1.5,7.50,1.75,4.329325472475174 km,"[-71.059648, 42.3538195]",https://42floors.com//us/ma/boston/38-chauncy-...
4,"201 Washington St, Boston, MA 02108, USA","5,980",17.25,9.0,10.0,-12.50,1.0,7.50,2.25,4.091570572360885 km,"[-71.058315, 42.3585846]",https://42floors.com//us/ma/boston/1-boston-pl...
...,...,...,...,...,...,...,...,...,...,...,...,...
63,"1 State St, New York, NY 10004, USA","2,040 - 4,600",0.00,0.0,0.0,-0.00,0.0,0.00,0.00,311.3277854543317 km,"[-74.0133207, 40.702703]",https://42floors.com//us/ma/boston/1-state-st?...
64,"207 South St, Elmhurst, IL 60126, USA","3,890",0.00,0.0,0.0,-0.00,0.0,0.00,0.00,1398.1919354859465 km,"[-87.93459089999999, 41.8857483]",https://42floors.com//us/ma/boston/207-south-s...
65,"41 Winter St, Kingston, MA 02364, USA","4,100",0.00,0.0,0.0,-0.00,0.0,0.00,0.00,43.9976810239303 km,"[-70.7503739, 42.0195674]",https://42floors.com//us/ma/boston/41-winter-s...
66,"1 Liberty St, New York, NY 10006, USA",500,0.00,0.0,0.0,-0.00,0.0,0.00,0.00,310.6742635751509 km,"[-74.01073939999999, 40.7098327]",https://42floors.com//us/ma/boston/1-liberty-s...


In [28]:
coordinates = int(input("Introduce the index of the office you want:" ))
selected_office = df["Location"].iloc[coordinates][::-1]

Introduce the index of the office you want:0


In [29]:
M.mapResults(selected_office,16,names,colors,collections,air_coord,selected_office,"final_map")

In [30]:
M.mappingChoice()

Do you want to see a map? (press enter to exit)y
Which map do you want to see? (initial_map / final_map): final_map


Do you want to see another map? (press enter to exit)
